# T-test
example

In [67]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from collections import Counter

In [ ]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [2]:
np.random.seed(99)

# Create sample data.
a = np.random.randn(10000)
b = np.random.randn(10000) + 0.1

# Use scipy.stats.ttest_ind.
t, p = ttest_ind(a, b, equal_var=False)

print('ttest_ind:' + str(p))

ttest_ind:2.81708330972e-07


# Cell-Type Populations in Plasma and PMA Treated

## Load Data

In [3]:
# Plasma Treated
net.load_file('../cytof_data/Plasma_UCT.txt')
net.random_sample(axis='row', num_samples=110000, random_state=99)
df_plasma = net.export_df()

print(df_plasma.shape)

# PMA Treated
net.load_file('../cytof_data/PMA_UCT.txt')
net.random_sample(axis='row', num_samples=110000, random_state=99)
df_pma = net.export_df()

print(df_pma.shape)

(110000, 28)
(110000, 28)


## Randomly Sample Data and Calc Cell-Type Populations

In [ ]:
np.random.seed(99)
num_samples = 2000

sampled_counts = {}

for i in range(100):

    # randomly subsample 
    random_state = int(np.random.random()*100)    
    net.load_df(df_plasma)
    net.random_sample(axis='row', num_samples=num_samples, random_state=random_state)
    inst_df = net.export_df()
    inst_rows = inst_df.index.tolist()

    # get cell-types from rows
    all_types = [i[2] for i in inst_rows]
    
    types_list = sorted(list(set(all_types)))
    
    # get the counts of all cell types in the subsampled data
    inst_counts = Counter(all_types)
    
    for inst_type in inst_counts.keys():
        
        # initialize counts
        if inst_type not in sampled_counts:
            sampled_counts[inst_type] = []
        
        # append new count
        sampled_counts[inst_type].append(inst_counts[inst_type])
    
    
    
sampled_counts['B cells']    

[131, 119]

['CD4 Tcells+CD27hi',
 'NK cells_CD16hi',
 'NK cells_CD16hi',
 'NK cells_CD16hi_CD57hi']